In [1]:
from sklearn.metrics import mean_absolute_error
from itertools import combinations
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer, MinMaxScaler,StandardScaler
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
import tensorflow
from tensorflow_addons.optimizers import AdamW
from keras import backend as K
from sklearn.model_selection import train_test_split
import os
import time

c:\Users\mj985\anaconda3\envs\assign\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


### 데이터 불러오기

- 최종적으로 데이터는 EDA과정을 feature를 생성한 값을 사용한다

- y는 log scale을 취했을 때에 평균적인 validation accuracy가 0.1 상승하는 것을 확인했음. - log 스케일로 진행
 : 과정에서 필요한 costom metric의 경우 exp 값을 취해 정상적으로 계산되게 만들어 주는 것이 필요.


In [2]:
df = pd.read_csv('final_model1.csv')

In [3]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [4]:
feature_cols = df.columns.tolist()
feature_cols.remove('Target')
target_cols = ['Target']
remove_list = ['Sex_I','Sex_F','Sex_M']
for col in remove_list:
    feature_cols.remove(col)
pipeline = Pipeline([('normalizer', Normalizer()),
                     ('scaler', StandardScaler())])

In [5]:
X = df.drop(['Target'], axis = 1)
y = df['Target']

# target 칼럼 log scale 적용
y = np.log10(y)

# train test split
X_train, X_test = train_test_split(X, test_size= 0.2, random_state = 42)
y_train, y_test = train_test_split(y, test_size= 0.2, random_state = 42)
X_train, X_val = train_test_split(X_train, test_size = 0.2, random_state = 42)
y_train, y_val = train_test_split(y_train, test_size = 0.2, random_state = 42)

# pipeline을 통해 normalize와 standard scaler 적용
X_train[feature_cols] = pipeline.fit_transform(X_train[feature_cols])
X_test[feature_cols] = pipeline.transform(X_test[feature_cols])
X_val[feature_cols] = pipeline.transform(X_val[feature_cols])

### 모델 구현

1. 인터넷 검색 모델

2. 최적의 파라미터 및 은닉층, 노드수 계산 모델

3. 최종 모델 선택 - 시간 고려(X) /// 추후 시간을 고려하는 모델과 성능을 고려하는 모델 두 가지 선택기능 구현 가능성

### 1. 인터넷 검색 모델

In [8]:
# 인터넷 구현 모델

# optimizer 구현 부분
schedule = tensorflow.optimizers.schedules.PiecewiseConstantDecay(
                [100, 150], [1e-0, 1e-1, 1e-2])
step = tf.Variable(0, trainable=False)
wd = lambda: 1e-3 * schedule(step)
optimizer = AdamW(learning_rate=0.001, weight_decay=wd)
def custom_opt(n):
    schedule = tensorflow.optimizers.schedules.PiecewiseConstantDecay([100, 150], [1e-0, 1e-1, 1e-2])
    step = tf.Variable(0, trainable=False)
    wd = lambda: 1e-3 * schedule(step)
    opt = AdamW(learning_rate = n, weight_decay = wd)
    return opt

# custom_metric 구현 - log scale 고려
def accuracy(y_true, y_pred):
    y_true = tf.math.exp(y_true)
    y_pred = tf.math.exp(y_pred)
    return 1 - tf.abs((y_true - y_pred) / y_true)

# 모델 구현
model = Sequential()
model.add(Dense(256, activation='elu', input_dim=X_train.shape[1]))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(32, activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))              
model.add(Dense(16, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1, activation='elu'))

# 모델 컴파일
model.compile(loss='mae', optimizer=optimizer , metrics=[accuracy])

# early stopping 구현추가
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, epochs=1000, batch_size=256, validation_data=(X_val, y_val), callbacks=[early_stopping])

print("====== Final_model =======")
print("train loss, train accuracy")
train_loss, train_acc = model.evaluate(X_train, y_train, verbose =2)

print("validation loss, validation accuracy")
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=2)

Epoch 1/1000
11/11 [==============================] - 3s 51ms/step - loss: 0.7977 - accuracy: 0.4371 - val_loss: 0.5152 - val_accuracy: 0.6054
Epoch 2/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.4312 - accuracy: 0.5830 - val_loss: 0.2965 - val_accuracy: 0.7509
Epoch 3/1000
11/11 [==============================] - 0s 12ms/step - loss: 0.3129 - accuracy: 0.6921 - val_loss: 0.2296 - val_accuracy: 0.8003
Epoch 4/1000
11/11 [==============================] - 0s 13ms/step - loss: 0.2407 - accuracy: 0.7709 - val_loss: 0.1843 - val_accuracy: 0.8354
Epoch 5/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.1950 - accuracy: 0.8101 - val_loss: 0.1630 - val_accuracy: 0.8527
Epoch 6/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.1718 - accuracy: 0.8325 - val_loss: 0.1417 - val_accuracy: 0.8699
Epoch 7/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.1548 - accuracy: 0.8470 - val_loss: 0.1281 - val_accuracy: 0.8819

In [18]:
# train accuracy : 0.9342 // validation accuracy 0.9341

### 2. 최적의 파라미터 및 은닉층, 노드수 계산 모델

- optimizer의 경우 속도 면에서는 rmsprop, sgd가 좋았고, adam은 성능 부분에서 더 좋았음. → learning rate를 조절

In [7]:
def custom_opt2(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt

best_lst = []

In [20]:
# a. Dropout 미사용 (순정)

# 최고의 모델 찾기 - 검증 데이터와 표준화 진행한 데이터로 성능 구현(dropout사용)
act_func = ['relu', 'tanh', 'sigmoid', 'elu']
batch_lst = [8, 32, 64, 128, 256]
opt_lst = [0.01, 0.009, 0.006, 0.003, 0.001, 0.0005]
best_accuracy = 0.0
best_hyperparams = {}
best_time = 11111.0
time_hyper = {}

for func in act_func:
    for batch in batch_lst:
        for opti in opt_lst:
            # 모델 구현
            model = Sequential()
            model.add(Dense(256, activation=func, input_dim=X_train.shape[1]))
            model.add(Dense(128, activation=func))
            model.add(Dense(64, activation=func))
            model.add(Dense(32, activation=func))
            model.add(Dense(16, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(1, activation=func))

            # 모델 컴파일
            model.compile(loss='mse', optimizer=custom_opt2(opti), metrics=[accuracy])

            # early stopping 구현 - 커스텀 정확도 기준
            early_stopping = EarlyStopping(monitor='val_accuracy', patience=10)
            start_time = time.time()
            model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping])
            end_time = time.time()
            cal_time = end_time - start_time
            loss, acc = model.evaluate(X_val, y_val, verbose=2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'batch_size': batch, 'learning_rate': opti}

            if cal_time < best_time:
                best_time = cal_time
                time_hyper = {'activation': func, 'batch_size': batch, 'learning_rate': opti}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
best_lst.append(best_accuracy)

print('Best time:', best_time)
print('time hyper_params:', time_hyper)



Epoch 1/1000
334/334 [==============================] - 2s 2ms/step - loss: 0.9751 - accuracy: 0.3799 - val_loss: 0.9579 - val_accuracy: 0.3828
Epoch 2/1000
334/334 [==============================] - 1s 2ms/step - loss: 0.9751 - accuracy: 0.3799 - val_loss: 0.9579 - val_accuracy: 0.3828
Epoch 3/1000
334/334 [==============================] - 1s 2ms/step - loss: 0.9751 - accuracy: 0.3799 - val_loss: 0.9579 - val_accuracy: 0.3828
Epoch 4/1000
334/334 [==============================] - 0s 1ms/step - loss: 0.9751 - accuracy: 0.3799 - val_loss: 0.9579 - val_accuracy: 0.3828
Epoch 5/1000
334/334 [==============================] - 0s 1ms/step - loss: 0.9751 - accuracy: 0.3799 - val_loss: 0.9579 - val_accuracy: 0.3828
Epoch 6/1000
334/334 [==============================] - 0s 1ms/step - loss: 0.9751 - accuracy: 0.3799 - val_loss: 0.9579 - val_accuracy: 0.3828
Epoch 7/1000
334/334 [==============================] - 0s 1ms/step - loss: 0.9751 - accuracy: 0.3799 - val_loss: 0.9579 - val_accuracy:

In [21]:
# b. Dropout 사용

# 최고의 모델 찾기 - 검증 데이터와 표준화 진행한 데이터로 성능 구현(dropout사용)
act_func = ['relu', 'tanh', 'sigmoid', 'elu']
batch_lst = [8, 32, 64, 128, 256]
opt_lst = [0.01, 0.009, 0.006, 0.003, 0.001, 0.0005]
best_accuracy = 0.0
best_hyperparams = {}
best_time = 11111.0
time_hyper = {}

for func in act_func:
    for batch in batch_lst:
        for opti in opt_lst:
            # 모델 구현
            model = Sequential()
            model.add(Dense(256, activation=func, input_dim=X_train.shape[1]))
            model.add(Dropout(0.2)) # Dropout 추가
            model.add(Dense(128, activation=func))
            model.add(Dropout(0.2)) # Dropout 추가
            model.add(Dense(64, activation=func))
            model.add(Dropout(0.2)) # Dropout 추가
            model.add(Dense(32, activation=func))
            model.add(Dense(16, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(1, activation=func))

            # 모델 컴파일
            model.compile(loss='mse', optimizer=custom_opt2(opti), metrics=[accuracy])

            # early stopping 구현 - 커스텀 정확도 기준
            early_stopping = EarlyStopping(monitor='val_accuracy', patience=10)
            start_time = time.time()
            model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping])
            end_time = time.time()
            cal_time = end_time - start_time
            loss, acc = model.evaluate(X_val, y_val, verbose=2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'batch_size': batch, 'learning_rate': opti}

            if cal_time < best_time:
                best_time = cal_time
                time_hyper = {'activation': func, 'batch_size': batch, 'learning_rate': opti}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
best_lst.append(best_accuracy)

print('Best time:', best_time)
print('time hyper_params:', time_hyper)



Epoch 1/1000
334/334 [==============================] - 2s 2ms/step - loss: 0.0342 - accuracy: 0.8840 - val_loss: 0.0109 - val_accuracy: 0.9246
Epoch 2/1000
334/334 [==============================] - 1s 2ms/step - loss: 0.0094 - accuracy: 0.9280 - val_loss: 0.0070 - val_accuracy: 0.9343
Epoch 3/1000
334/334 [==============================] - 1s 2ms/step - loss: 0.0085 - accuracy: 0.9299 - val_loss: 0.0089 - val_accuracy: 0.9222
Epoch 4/1000
334/334 [==============================] - 1s 2ms/step - loss: 0.0089 - accuracy: 0.9293 - val_loss: 0.0072 - val_accuracy: 0.9340
Epoch 5/1000
334/334 [==============================] - 1s 2ms/step - loss: 0.0092 - accuracy: 0.9273 - val_loss: 0.0083 - val_accuracy: 0.9308
Epoch 6/1000
334/334 [==============================] - 1s 2ms/step - loss: 0.0084 - accuracy: 0.9314 - val_loss: 0.0071 - val_accuracy: 0.9332
Epoch 7/1000
334/334 [==============================] - 1s 2ms/step - loss: 0.0083 - accuracy: 0.9311 - val_loss: 0.0071 - val_accuracy:

In [22]:
# c. Dropout, batchNormalize 사용

# 최고의 모델 찾기 - 검증 데이터와 표준화 진행한 데이터로 성능 구현(dropout사용)
act_func = ['relu', 'tanh', 'sigmoid', 'elu']
batch_lst = [8, 32, 64, 128, 256]
opt_lst = [0.01, 0.009, 0.006, 0.003, 0.001, 0.0005]
best_accuracy = 0.0
best_hyperparams = {}
best_time = 11111.0
time_hyper = {}

for func in act_func:
    for batch in batch_lst:
        for opti in opt_lst:
            # 모델 구현
            model = Sequential()
            model.add(Dense(256, activation=func, input_dim=X_train.shape[1]))
            model.add(BatchNormalization())  # BatchNormalization 추가
            model.add(Dropout(0.2))
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization())  # BatchNormalization 추가
            model.add(Dropout(0.2))
            model.add(Dense(64, activation=func))
            model.add(BatchNormalization())  # BatchNormalization 추가
            model.add(Dropout(0.2))
            model.add(Dense(32, activation=func))
            model.add(Dense(16, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(1, activation=func))

            # 모델 컴파일
            model.compile(loss='mse', optimizer=custom_opt2(opti), metrics=[accuracy])

            # early stopping 구현 - 커스텀 정확도 기준
            early_stopping = EarlyStopping(monitor='val_accuracy', patience=10)
            start_time = time.time()
            model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping])
            end_time = time.time()
            cal_time = end_time - start_time
            loss, acc = model.evaluate(X_val, y_val, verbose=2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'batch_size': batch, 'learning_rate': opti}

            if cal_time < best_time:
                best_time = cal_time
                time_hyper = {'activation': func, 'batch_size': batch, 'learning_rate': opti}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
best_lst.append(best_accuracy)

print('Best time:', best_time)
print('time hyper_params:', time_hyper)



Epoch 1/1000
334/334 [==============================] - 3s 3ms/step - loss: 0.1550 - accuracy: 0.8212 - val_loss: 0.0079 - val_accuracy: 0.9323
Epoch 2/1000
334/334 [==============================] - 1s 2ms/step - loss: 0.0111 - accuracy: 0.9188 - val_loss: 0.0086 - val_accuracy: 0.9330
Epoch 3/1000
334/334 [==============================] - 1s 2ms/step - loss: 0.0109 - accuracy: 0.9192 - val_loss: 0.0102 - val_accuracy: 0.9194
Epoch 4/1000
334/334 [==============================] - 1s 2ms/step - loss: 0.0104 - accuracy: 0.9219 - val_loss: 0.0114 - val_accuracy: 0.9058
Epoch 5/1000
334/334 [==============================] - 1s 2ms/step - loss: 0.0106 - accuracy: 0.9212 - val_loss: 0.0077 - val_accuracy: 0.9325
Epoch 6/1000
334/334 [==============================] - 1s 2ms/step - loss: 0.0102 - accuracy: 0.9226 - val_loss: 0.0079 - val_accuracy: 0.9346
Epoch 7/1000
334/334 [==============================] - 1s 2ms/step - loss: 0.0100 - accuracy: 0.9227 - val_loss: 0.0079 - val_accuracy:

In [9]:
# d. 모델 구조 변경 (다이아몬드)

# 최고의 모델 찾기 - 검증 데이터와 표준화 진행한 데이터로 성능 구현(dropout사용)
act_func = ['relu', 'tanh', 'sigmoid', 'elu']
batch_lst = [8, 32, 64, 128, 256]
opt_lst = [0.01, 0.009, 0.006, 0.003, 0.001, 0.0005]
best_accuracy = 0.0
best_hyperparams = {}
best_time = 11111.0
time_hyper = {}

for func in act_func:
    for batch in batch_lst:
        for opti in opt_lst:
            # 모델 구현
            model = Sequential()
            model.add(Dense(32, activation=func, input_dim=X_train.shape[1]))
            model.add(BatchNormalization())  # BatchNormalization 추가
            model.add(Dropout(0.2))
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization())  # BatchNormalization 추가
            model.add(Dropout(0.2))
            model.add(Dense(256, activation=func))
            model.add(BatchNormalization())  # BatchNormalization 추가
            model.add(Dropout(0.2))
            model.add(Dense(64, activation=func))
            model.add(Dense(32, activation=func))
            model.add(Dense(16, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(1, activation=func))

            # 모델 컴파일
            model.compile(loss='mse', optimizer=custom_opt2(opti), metrics=[accuracy])

            # early stopping 구현 - 커스텀 정확도 기준
            early_stopping = EarlyStopping(monitor='val_accuracy', patience=10)
            start_time = time.time()
            model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping])
            end_time = time.time()
            cal_time = end_time - start_time
            loss, acc = model.evaluate(X_val, y_val, verbose=2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'batch_size': batch, 'learning_rate': opti}

            if cal_time < best_time:
                best_time = cal_time
                time_hyper = {'activation': func, 'batch_size': batch, 'learning_rate': opti}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
best_lst.append(best_accuracy)

print('Best time:', best_time)
print('time hyper_params:', time_hyper)



Epoch 1/1000
334/334 [==============================] - 7s 5ms/step - loss: 0.0248 - accuracy: 0.8886 - val_loss: 0.0085 - val_accuracy: 0.9337
Epoch 2/1000
334/334 [==============================] - 1s 4ms/step - loss: 0.0117 - accuracy: 0.9178 - val_loss: 0.0081 - val_accuracy: 0.9273
Epoch 3/1000
334/334 [==============================] - 2s 5ms/step - loss: 0.0114 - accuracy: 0.9182 - val_loss: 0.0077 - val_accuracy: 0.9312
Epoch 4/1000
334/334 [==============================] - 1s 4ms/step - loss: 0.0113 - accuracy: 0.9186 - val_loss: 0.0096 - val_accuracy: 0.9198
Epoch 5/1000
334/334 [==============================] - 1s 4ms/step - loss: 0.0115 - accuracy: 0.9182 - val_loss: 0.0086 - val_accuracy: 0.9318
Epoch 6/1000
334/334 [==============================] - 1s 4ms/step - loss: 0.0109 - accuracy: 0.9203 - val_loss: 0.0082 - val_accuracy: 0.9282
Epoch 7/1000
334/334 [==============================] - 1s 4ms/step - loss: 0.0110 - accuracy: 0.9203 - val_loss: 0.0102 - val_accuracy:

### 3. 최종 모델 선택

In [11]:
# Best hyperparameters: {'activation': 'relu', 'batch_size': 128, 'learning_rate': 0.01}
# Best validation accuracy: 0.9382919073104858
# Best time: 4.589000701904297
# time hyper_params: {'activation': 'relu', 'batch_size': 128, 'learning_rate': 0.009}

# c. Dropout, batchNormalize 사용

# 최고의 모델 찾기 - 검증 데이터와 표준화 진행한 데이터로 성능 구현(dropout사용)

act_func = ['relu']
batch_lst = [128, 256]
opt_lst = [0.01, 0.009, 0.006, 0.003, 0.001]
best_accuracy = 0.0
best_hyperparams = {}
best_time = 11111.0
time_hyper = {}

for func in act_func:
    for batch in batch_lst:
        for opti in opt_lst:
            # 모델 구현
            model = Sequential()
            model.add(Dense(256, activation=func, input_dim=X_train.shape[1]))
            model.add(BatchNormalization())  # BatchNormalization 추가
            model.add(Dropout(0.2))
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization())  # BatchNormalization 추가
            model.add(Dropout(0.2))
            model.add(Dense(64, activation=func))
            model.add(BatchNormalization())  # BatchNormalization 추가
            model.add(Dropout(0.2))
            model.add(Dense(32, activation=func))
            model.add(Dense(16, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(1, activation=func))

            # 모델 컴파일
            model.compile(loss='mse', optimizer=custom_opt2(opti), metrics=[accuracy])

            # early stopping 구현 - 커스텀 정확도 기준
            early_stopping = EarlyStopping(monitor='val_accuracy', patience=10)
            start_time = time.time()
            model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping])
            end_time = time.time()
            cal_time = end_time - start_time
            loss, acc = model.evaluate(X_val, y_val, verbose=2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'batch_size': batch, 'learning_rate': opti}

            if cal_time < best_time:
                best_time = cal_time
                time_hyper = {'activation': func, 'batch_size': batch, 'learning_rate': opti}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
best_lst.append(best_accuracy)

print('Best time:', best_time)
print('time hyper_params:', time_hyper)



Epoch 1/1000
21/21 [==============================] - 2s 14ms/step - loss: 0.0784 - accuracy: 0.7936 - val_loss: 0.1337 - val_accuracy: 0.7117
Epoch 2/1000
21/21 [==============================] - 0s 5ms/step - loss: 0.0120 - accuracy: 0.9157 - val_loss: 0.0356 - val_accuracy: 0.8513
Epoch 3/1000
21/21 [==============================] - 0s 5ms/step - loss: 0.0099 - accuracy: 0.9241 - val_loss: 0.0139 - val_accuracy: 0.9131
Epoch 4/1000
21/21 [==============================] - 0s 5ms/step - loss: 0.0088 - accuracy: 0.9282 - val_loss: 0.0130 - val_accuracy: 0.9161
Epoch 5/1000
21/21 [==============================] - 0s 5ms/step - loss: 0.0087 - accuracy: 0.9293 - val_loss: 0.0110 - val_accuracy: 0.9230
Epoch 6/1000
21/21 [==============================] - 0s 5ms/step - loss: 0.0085 - accuracy: 0.9294 - val_loss: 0.0085 - val_accuracy: 0.9328
Epoch 7/1000
21/21 [==============================] - 0s 5ms/step - loss: 0.0082 - accuracy: 0.9306 - val_loss: 0.0078 - val_accuracy: 0.9344
Epoch

In [20]:
# Best hyperparameters: {'activation': 'relu', 'batch_size': 256, 'learning_rate': 0.009}

model = Sequential()
model.add(Dense(256, activation='relu', input_dim=X_train.shape[1]))
model.add(BatchNormalization())  # BatchNormalization 추가
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())  # BatchNormalization 추가
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='relu'))

# 모델 컴파일
model.compile(loss='mse', optimizer=custom_opt2(0.009), metrics=[accuracy])

# early stopping 구현 - 커스텀 정확도 기준
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20)
start_time = time.time()
model.fit(X_train, y_train, epochs=1000, batch_size=256, validation_data=(X_val, y_val), callbacks=[early_stopping])
end_time = time.time()
print("==== Final Model ====")
print("==== val los acc ====")
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=2)

Epoch 1/1000
11/11 [==============================] - 2s 23ms/step - loss: 0.1493 - accuracy: 0.5521 - val_loss: 0.5650 - val_accuracy: 0.4946
Epoch 2/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.0251 - accuracy: 0.8780 - val_loss: 0.3970 - val_accuracy: 0.5476
Epoch 3/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.0187 - accuracy: 0.8936 - val_loss: 0.2947 - val_accuracy: 0.5932
Epoch 4/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.0167 - accuracy: 0.9010 - val_loss: 0.1623 - val_accuracy: 0.6820
Epoch 5/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.0154 - accuracy: 0.9035 - val_loss: 0.1362 - val_accuracy: 0.7060
Epoch 6/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.0115 - accuracy: 0.9152 - val_loss: 0.1010 - val_accuracy: 0.7434
Epoch 7/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.0108 - accuracy: 0.9195 - val_loss: 0.0633 - val_accuracy: 0.7968
Epoch